<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/Huggingfacetutorial/CaptumLongformerSequenceClassificationPapers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [3]:
pip install transformers --quiet

In [4]:
pip install captum --quiet

In [5]:
pip install datasets --quiet

In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [7]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [10]:
def predict(inputs, position_ids=None):
    output = model(inputs,
                   position_ids=position_ids)
    return output.logits

In [11]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [12]:
max_length = 2048
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids


In [13]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-cc784554b797f843
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-cc784554b797f843/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [15]:
testval = 923
text = cogs402_ds['text'][testval]
label = cogs402_ds['labels'][testval]
print(label)

0


In [16]:
# testval = 923
# text = cogs402_ds2['text'][testval]
# label = cogs402_ds2['labels'][testval]
# print(label)

In [17]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None):
    preds = predict(inputs,
                   position_ids=position_ids
                   )
    return torch.softmax(preds, dim = 1)[:, 0] # for negative attribution, 
    #return torch.softmax(preds, dim = 1)[:, 1] #<- for positive attribution

In [18]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

In [19]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
# attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [20]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    n_steps=150,
                                    internal_batch_size = 2)

In [21]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

In [22]:
attributions_sum = summarize_attributions(attributions)

In [23]:
print(len(all_tokens))

2050


In [24]:
score = predict(input_ids)

In [25]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)[0]),
                        label,
                        text,
                        attributions_sum.sum(),       
                        all_tokens,
                        delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


In [26]:
def get_topk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [27]:
def get_botk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [28]:
import pandas as pd
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum)

df_high = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)]})

df_low = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(bot_words_start, bot_word_ind_start, bot_words_val_start)]})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

In [29]:
df_high

,"Word(Index), Attribution"
0,". (1781), 0.39"
1,". (1301), 0.19"
2,". (115), 0.12"
3,". (234), 0.09"
4,". (1040), 0.08"
5,". (278), 0.08"
6,". (392), 0.07"
7,"Ġperformance (149), 0.06"
8,") (251), 0.05"
9,"Ġwe (253), 0.05"


In [30]:
df_low

,"Word(Index), Attribution"
0,". (150), -0.79"
1,"Ġconstants (1536), -0.22"
2,". (185), -0.15"
3,", (1036), -0.13"
4,"Ġoptimizations (1024), -0.07"
5,". (1579), -0.06"
6,". (64), -0.05"
7,"ĠTo (151), -0.05"
8,". (390), -0.04"
9,"Ġthat (252), -0.04"


In [34]:
d = {"tokens":all_tokens, "attribution":attributions_sum[:2050].cpu()}

In [35]:
print(len(all_tokens), len(attributions_sum))

2050 2560


In [36]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [37]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False)[:10]
highest_attrib_tokens

,attribution
tokens,
Ġto,0.202355
.,0.103961
Ġwe,0.058096
Ġand,0.057790
Ġperformance,0.056754
Ġprograms,0.054941
Ġprogramming,0.054594
Ġprogram,0.054025
Ġby,0.052102


In [38]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution'])[:10]
lowest_attrib_tokens

,attribution
tokens,
Ġconstants,-0.216487
Ġthe,-0.084447
Ġoptimizations,-0.064167
",",-0.064031
Ġ,-0.049030
ĠTo,-0.044388
Ġthat,-0.034032
Ġproposed,-0.027000
t,-0.011565
